<a href="https://colab.research.google.com/github/Buterbrod/Deep-Learning-School-2021/blob/main/%D0%9F%D1%80%D0%B5%D0%B4%D1%81%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BE%D1%82%D1%82%D0%BE%D0%BA%D0%B0_%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D1%82%D0%B5%D0%BB%D0%B5%D0%B9_(%D0%BE%D1%81%D0%B5%D0%BD%D1%8C_2021).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предсказание оттока пользователей (весна 2021)

Это соревнование является домашним заданием к 3 модулю продвинутого потока DLS (весна 2021)
- https://www.kaggle.com/c/advanced-dls-spring-2021/overview

----

Вам предстоит **научиться моделировать отток клиентов телеком компании**. 

Эта задача очень важна на практике и алгоритмы для ее решения используются в реальных телеком компаниях, ведь если мы знаем, что клиент собирается уйти от нас, то мы попытаться удержать его, предложив какие-то бонусы.

**Дано:**

На странице с данными вы можете загрузить три файла:
- train.csv: файл с обучающей выборкой (содержит колонки признаков и целевую переменную)
- test.csv: файл с тестовой выборкой (содержит только колонки признаков).
- submission.csv: пример файла с предсказаниями

**Требуется:**

В качестве ответа вам необходимо загрузить предсказания вероятностей ухода клиентов для test.csv.

Названия признаков и целевой пременной есть в baseline ноутбуке. (baseline - базовое решение, с которым обычно сравниают более продвинутые алгоритмы.)

**Решение:**

## Шаг 1. Подключить библиотеки


In [13]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## Шаг 2. Подключить Kaggle к Colab и забрать файлы соревнования



Документация:
- [kaggle-api](https://github.com/Kaggle/kaggle-api)
- [инструкция из интернета](https://askdev.ru/q/ispolzovanie-naborov-dannyh-kaggle-v-google-colab-581252)

**Предварительные шаги:**
- зайти на https://www.kaggle.com
- войти в account
- API, Create New API Token
- сохранить JSON на диск для дальнейшего подключения

In [14]:
# Запустить и выбрать kaggle.json, ранее выгруженный файл с Kaggle account страницы
from google.colab import files
files.upload()
None

Saving kaggle.json to kaggle.json


In [15]:
# Убедимся, что kaggle.json файл загрузился
!ls -lha kaggle.json

-rw-r--r-- 1 root root 64 Oct  7 19:37 kaggle.json


In [16]:
# Установим клиент Kaggle API
!pip install -q kaggle

In [17]:
# Клиент Kaggle API ожидает файл kaggle.json в ~/.kaggle
# Переместим его туда
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Включим warning для Kaggle tool startup
!chmod 600 ~/.kaggle/kaggle.json

In [18]:
# Выведем список файлов соревнования advanced-dls-spring-2021
!kaggle competitions files advanced-dls-spring-2021

name             size  creationDate         
--------------  -----  -------------------  
submission.csv   14KB  2021-03-03 13:03:16  
test.csv        213KB  2021-03-03 13:03:16  
train.csv       648KB  2021-03-03 13:03:16  


In [19]:
# Загрузим все данные по соревнованию с Kaggle на Colab
!kaggle competitions download -p /data/advanced-dls-spring-2021 --force advanced-dls-spring-2021

  0% 0.00/648k [00:00<?, ?B/s]
100% 648k/648k [00:00<00:00, 42.2MB/s]
  0% 0.00/213k [00:00<?, ?B/s]
100% 213k/213k [00:00<00:00, 64.2MB/s]
  0% 0.00/14.4k [00:00<?, ?B/s]
100% 14.4k/14.4k [00:00<00:00, 30.0MB/s]


In [20]:
# Выведем список загруженных файлов
!ls /data/advanced-dls-spring-2021

submission.csv	test.csv  train.csv


## Шаг 3. Загрузить начальные данные и посмотреть на них

In [21]:
data_dir = "/data/advanced-dls-spring-2021/"

In [144]:
# train.csv - файл с обучающей выборкой
# содержит колонки признаков и целевую переменную
train_data = pd.read_csv(data_dir + "train.csv")
train_data.head()

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod,Churn
0,55,19.50,1026.35,Male,0,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,0
1,72,25.85,1872.2,Male,0,Yes,No,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),0
2,1,75.90,75.9,Male,0,No,No,Yes,No,Fiber optic,No,No,No,Yes,No,No,Month-to-month,Yes,Electronic check,1
3,32,79.30,2570,Female,1,Yes,No,Yes,Yes,Fiber optic,No,No,Yes,No,No,No,Month-to-month,No,Mailed check,0
4,60,115.25,6758.45,Female,0,Yes,Yes,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Credit card (automatic),0


In [146]:
# test.csv - файл с тестовой выборкой.
# содержит только колонки признаков
test_data = pd.read_csv(data_dir + "test.csv")
test_data.head()

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod
0,42,56.10,2386.85,Male,0,Yes,No,Yes,Yes,DSL,Yes,No,No,No,No,No,One year,No,Credit card (automatic)
1,29,98.50,3004.15,Male,1,Yes,No,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check
2,9,76.25,684.85,Male,0,No,No,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check
3,67,74.00,4868.4,Female,0,Yes,Yes,Yes,Yes,DSL,Yes,Yes,No,Yes,Yes,No,Two year,No,Credit card (automatic)
4,71,20.10,1389.6,Female,0,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic)


In [142]:
#submission.csv -  файла с предскзааниями
subm_data = pd.read_csv(data_dir + "submission.csv")
print('Размерность: '+str(subm_data.shape))
subm_data.head()

Размерность: (1761, 2)


,Id,Churn
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [153]:
print('Размерность тренировочных данных: '+str(train_data.shape))
print('Размерность тестовых данных: '+str(test_data.shape))
print('Всего данных: ' + str(train_data.shape[0]) + ' + ' + str(test_data.shape[0]) +
      ' = ' + str((train_data.shape[0]) + test_data.shape[0]))

Размерность тренировочных данных: (5282, 20)
Размерность тестовых данных: (1761, 19)
Всего данных: 5282 + 1761 = 7043


### Шаг 4. Разбор и преобразование дата модели

In [154]:
# Уберем из модели строки с NULL значениями
train_data = train_data.dropna()
test_data = test_data.dropna()

# Выделим данные результата
result_data = train_data[train_data.columns[-1]]

# Объединим массив тестовых и проверочных данных, чтобы работать одинаково с массивами
all_data = train_data[train_data.columns[:-1]]
all_data['IsTrainData'] = 1
a = test_data
a['IsTrainData'] = 0
all_data = pd.concat([all_data, a], axis=0)

# Разделим поля по типам: численные / категориальные
numeric_indeces = np.array([0, 1, 2, 4, 19])
categorical_indeces = np.array([3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18])

# Положим их в соответствующие массивы
numeric_data = all_data[all_data.columns[numeric_indeces]]
categorical_data = all_data[all_data.columns[categorical_indeces]]

all_data.head()

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod,IsTrainData
0,55,19.50,1026.35,Male,0,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,1
1,72,25.85,1872.2,Male,0,Yes,No,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),1
2,1,75.90,75.9,Male,0,No,No,Yes,No,Fiber optic,No,No,No,Yes,No,No,Month-to-month,Yes,Electronic check,1
3,32,79.30,2570,Female,1,Yes,No,Yes,Yes,Fiber optic,No,No,Yes,No,No,No,Month-to-month,No,Mailed check,1
4,60,115.25,6758.45,Female,0,Yes,Yes,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Credit card (automatic),1


In [155]:
# Выведем численные данные
numeric_data.head()

,ClientPeriod,MonthlySpending,TotalSpent,IsSeniorCitizen,IsTrainData
0,55,19.50,1026.35,0,1
1,72,25.85,1872.2,0,1
2,1,75.90,75.9,0,1
3,32,79.30,2570,1,1
4,60,115.25,6758.45,0,1


In [156]:
# Выведем категориальные данные
categorical_data.head()

,Sex,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod
0,Male,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check
1,Male,Yes,No,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic)
2,Male,No,No,Yes,No,Fiber optic,No,No,No,Yes,No,No,Month-to-month,Yes,Electronic check
3,Female,Yes,No,Yes,Yes,Fiber optic,No,No,Yes,No,No,No,Month-to-month,No,Mailed check
4,Female,Yes,Yes,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Credit card (automatic)


In [157]:
# Выведем данные категориальных столбцов
for column_name in categorical_data.columns:
    print(len(test_data[column_name].unique()), column_name + ':', test_data[column_name].unique())

2 Sex: ['Male' 'Female']
2 HasPartner: ['Yes' 'No']
2 HasChild: ['No' 'Yes']
2 HasPhoneService: ['Yes' 'No']
3 HasMultiplePhoneNumbers: ['Yes' 'No' 'No phone service']
3 HasInternetService: ['DSL' 'Fiber optic' 'No']
3 HasOnlineSecurityService: ['Yes' 'No' 'No internet service']
3 HasOnlineBackup: ['No' 'Yes' 'No internet service']
3 HasDeviceProtection: ['No' 'Yes' 'No internet service']
3 HasTechSupportAccess: ['No' 'Yes' 'No internet service']
3 HasOnlineTV: ['No' 'Yes' 'No internet service']
3 HasMovieSubscription: ['No' 'Yes' 'No internet service']
3 HasContractPhone: ['One year' 'Month-to-month' 'Two year']
2 IsBillingPaperless: ['No' 'Yes']
4 PaymentMethod: ['Credit card (automatic)' 'Electronic check' 'Bank transfer (automatic)'
 'Mailed check']


In [158]:
# применим к ним One-Hot Encoding
dummies = pd.get_dummies(categorical_data)
print('Размер категориальных данных до: '+str(categorical_data.shape)+', после One-Hot Encoding: '+str(dummies.shape))

Размер категориальных данных до: (7043, 15), после One-Hot Encoding: (7043, 41)


In [160]:
# объединим преобразованные после One-Hot Encoding данные вместе
all_data = pd.concat([numeric_data, dummies], axis=1)
all_data.shape

(7043, 46)